<a href="https://colab.research.google.com/github/joony0512/Deep_Learning_Class/blob/main/Part6/P6_Ch03_CH01_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 오픈 도메인 대화시스템 실습
## 생성기반 방식 TEXT -> Encoder -> Decoder -> TEXT (End2End)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install sentencepiece
!pip install pytorch-crf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.2 MB/s eta 0:00:00


In [3]:
import re
import torch

%cd /content/drive/MyDrive/딥러닝_정주행/P6_Ch03.자연어처리실습_챗봇
from src.model import Tformer, save
from src.dataset import Preprocessing, MakeDataset
%cd /root

/content/drive/MyDrive/딥러닝_정주행/P6_Ch03.자연어처리실습_챗봇
/root


In [4]:
class E2E_dialog:
    def __init__(self, dataset, model_path):
        self.vocab = dataset.transformers_tokenizer
        self.vocab_size = dataset.transformers_tokenizer.vocab_size()

        self.model = Tformer(num_tokens=self.vocab_size, dim_model=256, num_heads=8, dff=512, num_layers=2, dropout_p=0.1)
        # device = torch.device('cpu')
        self.model.load_state_dict(torch.load(model_path, map_location=device))
        self.model.eval()
        self.MAX_LENGTH = 50

    def preprocess_sentence(self, sentence):
        sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
        sentence = sentence.strip()
        return sentence

    def evaluate(self, sentence):
        sentence = self.preprocess_sentence(sentence)
        input = torch.tensor([[2] + self.vocab.encode_as_ids(sentence) + [3]])
        output = torch.tensor([[2]])

        # 디코더의 예측 시작
        ps = []
        for i in range(self.MAX_LENGTH):
            src_mask = self.model.generate_square_subsequent_mask(input.shape[1])
            tgt_mask = self.model.generate_square_subsequent_mask(output.shape[1])

            src_padding_mask = self.model.gen_attention_mask(input)
            tgt_padding_mask = self.model.gen_attention_mask(output)

            predictions = self.model(input, output, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask).transpose(0,1)
            # 현재(마지막) 시점의 예측 단어를 받아온다.
            predictions = predictions[:, -1:, :]
            predictions = torch.softmax(predictions.view(-1).cpu(), dim=0)
            predictions = torch.max(predictions, axis = -1)
            predicted_p = predictions.values
            ps.append(predicted_p)
            predicted_id =predictions.indices.view(1,1)


            # 만약 마지막 시점의 예측 단어가 종료 토큰이라면 예측을 중단
            if torch.equal(predicted_id[0][0], torch.tensor(3)):
                break

            # 마지막 시점의 예측 단어를 출력에 연결한다.
            # 이는 for문을 통해서 디코더의 입력으로 사용될 예정이다.
            output = torch.cat([output, predicted_id], axis=1)

        return torch.squeeze(output, axis=0).cpu().numpy(), (sum(ps)/len(ps)).detach().numpy()

    def predict(self, sentence):
        prediction, predicted_sentence_p = self.evaluate(sentence)
        predicted_sentence = self.vocab.Decode(list(map(int,[i for i in prediction if i < self.vocab_size])))

        print('Input: {}'.format(sentence))
        print('Output: {}'.format(predicted_sentence))

        return predicted_sentence, predicted_sentence_p

In [5]:
chitchat_pretrain_path = "/content/drive/MyDrive/딥러닝_정주행/P6_Ch03.자연어처리실습_챗봇/data/pretraining/save/4_chitchat_transformer_model/chitchat_transformer_1.215381_steps_81.pt"

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dataset = MakeDataset()
e2e = E2E_dialog(dataset,chitchat_pretrain_path)

In [13]:
%%time
s, p = e2e.predict("난 뭘 해야 할까?")

Input: 난 뭘 해야 할까?
Output: 모르는 사실에 더 화가 나 있을 거예요.
CPU times: user 135 ms, sys: 0 ns, total: 135 ms
Wall time: 156 ms


In [9]:
float(p)

0.40460020303726196